<a href="https://colab.research.google.com/github/felix0097/CVAE_mnist/blob/master/cvae_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install tensorflow==2.1.0
import tensorflow as tf

      Successfully uninstalled tensorflow-1.15.0


In [5]:
tf.__version__

'1.15.0'